In [27]:
import pandas as pd
import os
import datasets

import pyarrow as pa
import pyarrow.parquet as pq

from collections import defaultdict

from tqdm import tqdm

import numpy as np

import gc

In [2]:
pmid_to_cc = pd.read_csv('../assets/pmid/PMC-ids.csv')
pmid_to_cc2 = pd.read_csv('../assets/pmid/oa_non_comm_use_pdf.csv')
pmid_to_cc3 = pd.read_csv('../assets/pmid/oa_file_list.csv')
pmid_to_cc4 = pd.read_csv('../assets/pmid/oa_comm_use_file_list.csv')

/var/folders/_s/tsq34rsj2q56tb1d67d839sh0000gn/T/ipykernel_78353/648927490.py:1: DtypeWarning: Columns (4,5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  pmid_to_cc = pd.read_csv('../assets/pmid/PMC-ids.csv')


In [3]:
pmid_to_cc.columns

Index(['Journal Title', 'ISSN', 'eISSN', 'Year', 'Volume', 'Issue', 'Page',
       'DOI', 'PMCID', 'PMID', 'Manuscript Id', 'Release Date'],
      dtype='object')

In [4]:
pmid_to_cc2.columns

Index(['File', 'Article Citation', 'Accession ID',
       'Last Updated (YYYY-MM-DD HH:MM:SS)', 'PMID', 'License'],
      dtype='object')

In [5]:
pmid_to_cc3.columns

Index(['File', 'Article Citation', 'Accession ID',
       'Last Updated (YYYY-MM-DD HH:MM:SS)', 'PMID', 'License'],
      dtype='object')

In [6]:
pmid_to_cc4.columns

Index(['File', 'Article Citation', 'Accession ID',
       'Last Updated (YYYY-MM-DD HH:MM:SS)', 'PMID', 'License'],
      dtype='object')

In [7]:
# Combine all mappings into a single DataFrame
pmid_to_cc_combined = pd.concat([pmid_to_cc2, pmid_to_cc3, pmid_to_cc4], ignore_index=True)
pmid_to_cc_combined.drop_duplicates(subset=['Accession ID'], inplace=True)

In [8]:
pmid_to_cc_combined.dropna(subset=['PMID'], inplace=True)

In [9]:
pmid_to_cc_combined.groupby('License').size() / len(pmid_to_cc_combined)

License
CC BY          0.654906
CC BY-NC       0.117903
CC BY-NC-ND    0.121303
CC BY-NC-SA    0.040075
CC BY-ND       0.001527
CC BY-SA       0.000260
CC0            0.018005
NO-CC CODE     0.046021
dtype: float64

In [10]:
pmid_to_cc_combined.PMID = pmid_to_cc_combined.PMID.astype(str).str.strip()
pmid_to_cc_combined.PMID = pmid_to_cc_combined.PMID.astype('float')
pmid_to_cc_combined.PMID = pmid_to_cc_combined.PMID.astype('int64')

In [11]:
pmid_to_cc_combined[['PMID', 'License']].drop_duplicates().License.value_counts(dropna=False)

License
CC BY          4670865
CC BY-NC-ND     865149
CC BY-NC        840898
NO-CC CODE      328225
CC BY-NC-SA     285822
CC0             128411
CC BY-ND         10888
CC BY-SA          1855
Name: count, dtype: int64

In [12]:
pmid_to_cc_2 = pd.read_parquet('../assets/pmid/PMC_LICENSE.parquet')
pmid_to_cc_2 = pmid_to_cc_2.query('PMID>0')

pmid_to_cc = pd.concat([pmid_to_cc_combined[['PMID', 'License']], pmid_to_cc_2[['PMID', 'License']]], axis=0)

In [13]:
pmid_to_cc[['PMID', 'License']].drop_duplicates().License.value_counts(dropna=False)

License
CC BY          4674709
CC BY-NC-ND     865803
CC BY-NC        841035
NO-CC CODE      329735
CC BY-NC-SA     286209
CC0             128416
CC BY-ND         10894
CC BY-SA          1856
Name: count, dtype: int64

In [14]:
# load parquet 
abstracts = pd.read_parquet('../assets/abstracts_nl.parquet')

In [15]:
abstracts.head()

,pmid,year,txt_line,text
0,5038324,2019,0,Het effect van verschillende glycosidasen op d...
1,5038325,2019,1,Negentien gevallen van maligne lymfomata werde...
2,5038326,2019,2,Transplantatie-immuniteit tegen adenovirus 12-...
3,5038327,2019,3,Het remmende effect van een i.v.- of i.p.-inje...
4,5038868,2019,4,Afscheiding en afbraak van rode bloedcellen (R...


In [16]:
abstracts = abstracts.merge(pmid_to_cc[['PMID', 'License']], how='left', left_on='pmid', right_on='PMID').drop('PMID', axis=1)

In [ ]:
abstracts.License.value_counts(dropna=False)
abstracts = abstracts.assign(License=abstracts.License.fillna('NO-CC CODE'))

License
NaN            13225762
CC BY           4228708
CC BY-NC         527167
CC BY-NC-ND      456290
CC BY-NC-SA      322529
NO-CC CODE       320287
CC0               44201
CC BY-ND           8806
CC BY-SA           1128
Name: count, dtype: int64

In [24]:
abstracts = abstracts.drop_duplicates(subset=['txt_line'])

In [ ]:
abstracts.to_parquet('../assets/abstracts_with_license.parquet', )

In [40]:
NUM_PARTITIONS=20

In [41]:
abstracts = abstracts.assign(partition=np.random.randint(0,NUM_PARTITIONS, size=len(abstracts)))

In [42]:
table = pa.Table.from_pandas(abstracts)

pq.write_to_dataset(
    table,
    root_path='abstracts_with_license.parquet',
    partition_cols=['partition'],
)